In [ ]:
from mt import *

In [ ]:
# The directory containing the images
data_dir = '/Volumes/martin/A08_A09_test/folder/modified'

# glob string for images
im_glob = os.path.join(data_dir, 'Round1_max_composite_bgsub.tif')
im_glob_405 = os.path.join(data_dir, 'Round1_max_composite_405.tif')
im_glob_488 = os.path.join(data_dir, 'Round1_max_composite_488_**_Simple Segmentation.tiff')
im_glob_561 = os.path.join(data_dir, 'Round1_max_composite_561_**_Simple Segmentation.tiff')
im_glob_594 = os.path.join(data_dir, 'Round1_max_composite_594_**_Simple Segmentation.tiff')
im_glob_633 = os.path.join(data_dir, 'Round1_max_composite_633_**_Simple Segmentation.tiff')

# Get list of images
im_list = sorted(glob.glob(im_glob))
im_list_405 = sorted(glob.glob(im_glob_405))
im_list_488 = sorted(glob.glob(im_glob_488))
im_list_561 = sorted(glob.glob(im_glob_561))
im_list_594 = sorted(glob.glob(im_glob_594))
im_list_633 = sorted(glob.glob(im_glob_633))

In [ ]:
im_list

In [ ]:
im_list_405

In [ ]:
if len(im_list_488) & len(im_list_561) & len(im_list_594) & len(im_list_633) == 48:
    print('Length of each channel is 48.')
else:
    print('error')
    print('488:', len(im_list_488))
    print('561:', len(im_list_561))
    print('594:', len(im_list_594))
    print('633:', len(im_list_633))

bgsub has rolling ball of 50 applied.

dapi has no bgsub applied.

Each .tif file has 72 pictures. There are 6 conditions with 12 positions each. The first two conditions are irrelevant. For the last four, there is the A8 ABE, A8 pUC19, A9 ABE, and A9 pUC19.

There are four readout probes. Generally they correspond to 488 and 561 being for unedit barcode and 594 and 647 being for edited barcode.

In [ ]:
# Read data using skimage
im = skimage.io.imread(im_list[0])
im_405 = skimage.io.imread(im_list_405[0])

In [ ]:
im.shape

In [ ]:
# Slice out the first two conditions
im = im[24:]
im_405 = im_405[24:]

Right now, I have an intensity image that has bg subtraction (with all the channels), dapi image, and binary masks for each channel (4 total). 

The conditions are as follows:

**A8_ABE**: i=0-11

**A8_pUC19**: i=12-23

**A9_ABE**: i=24-35

**A9_pUC19**: i=36-47

From the intensity image, I will separate the different channels for the intensity image.

In [ ]:
i = 37

In [ ]:
af488 = im[i][0]
af561 = im[i][1]
af594 = im[i][2]
af633 = im[i][3]
dapi = im_405[i]

In [ ]:
# Read data using skimage
im488 = skimage.io.imread(im_list_488[i])
im561 = skimage.io.imread(im_list_561[i])
im594 = skimage.io.imread(im_list_594[i])
im633 = skimage.io.imread(im_list_633[i])

In [ ]:
def register_images(cfp, chfp):
    '''
    Registers the two channels using register_translation from skimage.features.
    Prints detected pixel offset (y,x). If shift is more than 10 pixels in y or x, will default to [y, x] = [0, -5]
    Returns corrected moving array.
    
    ----------
    Parameters
    ----------
    cfp : array
        best slice of cfp
    chfp : array
        best slice of chfp
    
    -------
    Output
    -------
    chfp : array
        fixed chfp slice
    '''
    # pixel precision first
    shift, error, diffphase = register_translation(cfp, chfp)
    print(f"Detected pixel offset (y, x): {shift}")
    
    if abs(shift[0]) > 10.0:
        shift = [0,0]
        print('detected pixel offset is more than 10, default shift to [0,0].')
    elif abs(shift[1]) > 10.0:
        shift = [0,0]
        print('detected pixel offset is more than 10, default shift to [0,0].')
    else:
        print('pass check: shift is less than 10 in both dimensions')
        
    chfp_shift = ndi.shift(chfp, shift)

    # corrected chfp_slice
    chfp = chfp_shift
    return chfp, shift

In [ ]:
# shift the intensity images according to 488 channel
af561, shift561 = register_images(af488, af561)
af594, shift594 = register_images(af488, af594)
af633, shift633 = register_images(af488, af633)

In [ ]:
# apply the same shift from the intensity image to the binary image
im561 = ndi.shift(im561, shift561)
im594 = ndi.shift(im594, shift594)
im633 = ndi.shift(im633, shift633)

In [ ]:
def imadjust(img, lower_bound=0.05, upper_bound=99.95):
    lower = np.percentile(img, lower_bound)
    upper = np.percentile(img, upper_bound)
    out = (img - lower) * (255 / (upper - lower))
    return np.clip(out, 0, 255, out)

In [ ]:
af488_adj = imadjust(af488)
af561_adj = imadjust(af561)
af594_adj = imadjust(af594)
af633_adj = imadjust(af633)

In [ ]:
dapi_g = skimage.filters.gaussian(dapi, 1.5)

In [ ]:
threshold = threshold_otsu(dapi_g)
print(threshold, 'is where the cutoff point is.')
dapi_bw = dapi_g > threshold

In [ ]:
dapi_labels, df_dapi, dapi_area, dapi_distance, dapi_markers = ws(dapi_bw, 2, np.ones((45,45)), 2000, dapi)

In [ ]:
bokeh.io.show(show_three_ims(dapi, dapi_bw, dapi_labels,
                             cmap=[None, None, colorcet.b_glasbey_hv]))


In [ ]:
dapi_labels.max()

In [ ]:
im488 = im488 == 1
im561 = im561 == 1
im594 = im594 == 1
im633 = im633 == 1

In [ ]:
label_im488 = skimage.measure.label(im488, connectivity=1)
label_im561 = skimage.measure.label(im561, connectivity=1)
label_im594 = skimage.measure.label(im594, connectivity=1)
label_im633 = skimage.measure.label(im633, connectivity=1)

In [ ]:
size=2
label_im488 = skimage.morphology.remove_small_objects(label_im488, min_size=size)
label_im561 = skimage.morphology.remove_small_objects(label_im561, min_size=size)
label_im594 = skimage.morphology.remove_small_objects(label_im594, min_size=size)
label_im633 = skimage.morphology.remove_small_objects(label_im633, min_size=size)

In [ ]:
label_im488.max()

In [ ]:
# Show result
bokeh.io.show(show_six_ims(af488_adj, im488, label_im488, af594_adj, im594, label_im594,
                            titles=["488", "488", "488", "594", "594", "594"],
                            cmap=[None, None, colorcet.b_glasbey_hv, None, None, colorcet.b_glasbey_hv]))

In [ ]:
# Show result
bokeh.io.show(show_six_ims(af561_adj, im561, label_im561, af633_adj, im633, label_im633,
                            titles=["561", "561", "561", "633", "633", "633"],
                            cmap=[None, None, colorcet.b_glasbey_hv, None, None, colorcet.b_glasbey_hv]))

In [ ]:
im488_594_binary = np.logical_or(label_im488, label_im594)
im561_633_binary = np.logical_or(label_im561, label_im633)

im488_594 = skimage.measure.label(im488_594_binary, connectivity=1)
im561_633 = skimage.measure.label(im561_633_binary, connectivity=1)

bokeh.io.show(show_six_ims(label_im488, label_im594, im488_594, af488_adj, af594_adj, im488_594_binary,
                             titles=['488', '594', 'or', '488', '594', 'or'], 
                             cmap=[colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, None, None, None]))

In [ ]:
im488_594 = skimage.morphology.remove_small_objects(im488_594, min_size=size)
im561_633 = skimage.morphology.remove_small_objects(im561_633, min_size=size)

In [ ]:
selem = skimage.morphology.disk(1)

# Dilate image
im488_594 = skimage.morphology.dilation(im488_594, selem)
im561_633 = skimage.morphology.dilation(im561_633, selem)

bokeh.io.show(show_five_ims(label_im561, label_im633, im561_633, af561_adj, af633_adj,
                             titles=['561', '633', 'or', '561', '633'], 
                             cmap=[colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, None, None]))

In [ ]:
area_488_594 = skimage.measure.regionprops_table(im488_594, properties=('label',
                                                                        'centroid',
                                                                        'area',
                                                                        ))
df_area_488_594 = pd.DataFrame(area_488_594)

In [ ]:
df_counts = df_area_488_594['area'].value_counts(
).reset_index(
).rename(
    columns={"index": "area", "area": "count"}
)

# Take a look
df_counts

In [ ]:
hv.Spikes(
    data=df_counts,
    kdims=['area'],
    vdims=['count']
)

In [ ]:
im488_594.max()

In [ ]:
im488_594_filt = im488_594 * dapi_bw
im561_633_filt = im561_633 * dapi_bw

In [ ]:
def coloc(large_sub, unedited, edited):
    '''
    Function to get all nuclei with mean intensity for each channel. Can do two channels.
    
    ---------
    Parameters
    ---------
    large_sub : array
        OR watershed image with small and large segments subtracted
    unedited : array
        raw intensity image
    edited : array
        raw intensity image
    
    -------
    Output
    -------
    df_co : DataFrame
        DataFrame with all dots with mean intensity for each channel.
    '''

    props_unedited = skimage.measure.regionprops_table(large_sub, intensity_image=unedited, properties=('label',
                                                                                             'centroid',
                                                                                             'area',
                                                                                             'mean_intensity'))
    df_unedited = pd.DataFrame(props_unedited)
    df_unedited['sum_intensity'] = df_unedited['area']*df_unedited['mean_intensity']
    

    props_edited = skimage.measure.regionprops_table(large_sub, intensity_image=edited, properties=('label',
                                                                                                 'centroid',
                                                                                                 'area',
                                                                                                 'mean_intensity'))
    df_edited = pd.DataFrame(props_edited)
    df_edited['sum_intensity'] = df_edited['area']*df_edited['mean_intensity']
    
    label_vals = df_unedited['label'].values
    
    mean_unedited_vals = df_unedited['mean_intensity'].values
    mean_edited_vals = df_edited['mean_intensity'].values
    
    sum_unedited_vals = df_unedited['sum_intensity'].values
    sum_edited_vals = df_edited['sum_intensity'].values
    
    data = {'label': label_vals,
            'Unedited mean intensity': mean_unedited_vals, 
            'Edited mean intensity': mean_edited_vals,
            'Unedited sum intensity': sum_unedited_vals,
            'Edited sum intensity': sum_edited_vals}
    df_co = pd.DataFrame(data)
    return df_co, df_unedited, df_edited

In [ ]:
#odd number round
#df_bc1, df_bc1_unedited, df_bc1_edited = coloc(im488_594_filt, af488, af594)
#df_bc2, df_bc2_unedited, df_bc2_edited = coloc(im561_633_filt, af561, af633)

#even number round
df_bc1, df_bc1_unedited, df_bc1_edited = coloc(im488_594_filt, af594, af488)
df_bc2, df_bc2_unedited, df_bc2_edited = coloc(im561_633_filt, af633, af561)

In [ ]:
df_bc1['ln(intensity) of unedited probe'] = np.log(df_bc1['Unedited sum intensity'])
df_bc1['ln(intensity) of edited probe'] = np.log(df_bc1['Edited sum intensity'])
df_bc2['ln(intensity) of unedited probe'] = np.log(df_bc2['Unedited sum intensity'])
df_bc2['ln(intensity) of edited probe'] = np.log(df_bc2['Edited sum intensity'])

In [ ]:
df_bc1['position'] = i
df_bc2['position'] = i

In [ ]:
df_bc1

In [ ]:
df_bc1.loc[df_bc1['Unedited sum intensity'] != 0]

In [ ]:
hv.Points(data=df_bc1,
          kdims=['Unedited mean intensity', 'Edited mean intensity'],
          label='bc1'
)

In [ ]:
hv.Points(data=df_bc1,
          kdims=['Unedited sum intensity', 'Edited sum intensity'],
          label='bc1'
)

In [ ]:
hv.Points(data=df_bc1.loc[(df_bc1['Unedited sum intensity'] != 0) & (df_bc1['Edited sum intensity'] != 0)],
          kdims=['ln(intensity) of unedited probe', 'ln(intensity) of edited probe'],
          label='bc1'
)

In [ ]:
hv.Points(data=df_bc2,
          kdims=['Unedited sum intensity', 'Edited sum intensity'],
          label='bc2'
)

In [ ]:
hv.Points(data=df_bc2.loc[(df_bc2['Unedited sum intensity'] != 0) & (df_bc2['Edited sum intensity'] != 0)],
          kdims=['Unedited sum intensity', 'Edited sum intensity'],
          label='bc2'
).opts(
    logx=True,
    logy=True)

In [ ]:
hv.Points(data=df_bc2.loc[(df_bc2['Unedited sum intensity'] != 0) & (df_bc2['Edited sum intensity'] != 0)],
          kdims=['ln(intensity) of unedited probe', 'ln(intensity) of edited probe'],
          label='bc2'
)

In [ ]:
df_bc1.loc[df_bc1['ln(intensity) of unedited probe'] < 8]['label']

In [ ]:

check = np.zeros_like(im488_594_filt)
for i, label in enumerate(df_bc1.loc[df_bc1['ln(intensity) of unedited probe'] < 8]['label']):
    x = im488_594_filt == label
    y = x*1
    check = check + y

In [ ]:

# Show result

bokeh.io.show(show_six_ims(af488_adj, af594_adj, check, label_im488, label_im594, im488_594,
                            titles=["488", "594", "check", "488", "594", "or"],
                            cmap=[None, None, None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv,]))


In [ ]:
unedited = np.zeros_like(im488_594_filt)
edited = np.zeros_like(im488_594_filt)

for i, label in enumerate(df_bc1['label']):
    x = im488_594_filt == label
    
    y = x*df_bc1['Unedited sum intensity'].values[i]
    unedited = unedited + y
    
    z = x*df_bc1['Edited sum intensity'].values[i]
    edited = edited + z

In [ ]:
hv.Image(unedited).opts(
    tools=['hover'],
    width=500,
    height=500,
    cmap='plasma')

In [ ]:
hv.Image(edited).opts(
    tools=['hover'],
    width=500,
    height=500,
    cmap='plasma')